In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from joblib import parallel_backend
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
abc_data = pd.read_csv('../2_Labelled_Data/bert_label.csv')
abc_data.head()

publish_date                                      headline_text bert_label
0   24/01/2019  [('malaysia', 'appoint', 'new'), ('appoint', '...   positive
1   21/01/2019  [('kurti', 'patterson', 'rush'), ('patterson',...    neutral
2   31/01/2019  [('katherin', 'darwin', 'lash'), ('darwin', 'l...    neutral
3   26/01/2019  [('spirit', 'matthew', 'flinder'), ('matthew',...   positive
4   31/01/2019  [('nsw', 'govern', 'negoti'), ('govern', 'nego...   positive

### Feature Extraction and Data Splitting into Train, Test and Validation

In [3]:
# Remove target variable from dataset
target = ['bert_label']
features = ['headline_text']

X = abc_data[features[0]].values # Get predictor variable (headline_text column)
y = abc_data[target[0]].values # Get target variable (sentiment column)

# Split the data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# Perform TF normalization on the text data
tf = TfidfVectorizer(norm='l2')
X_train = tf.fit_transform(X_train)
X_test = tf.transform(X_test)

# Split the test data into test and validation datasets
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0, stratify=y_test)

# Output dimensions
print('Train data shape: ', X_train.shape)
print('Test data shape: ', X_test.shape)
print('Validation data shape: ', X_val.shape)


Train data shape:  (7257, 8380)
Test data shape:  (907, 8380)
Validation data shape:  (908, 8380)


### NB

In [4]:
# Define the parameter grid for hyperparameter tuning
mnb_param_grid = {
    'alpha': [0.1, 1, 10],
    'fit_prior': [True, False]
}

# Create a repeated 5-fold cross-validation object
kf = KFold(n_splits=5)

# Create a MultinomialNB classifier
mnb = MultinomialNB()

# Use GridSearchCV for repeated 5-fold cross validation and hyperparameter tuning
mnb_grid_search = GridSearchCV(mnb, mnb_param_grid, cv=kf, n_jobs=-1)

with parallel_backend('multiprocessing', n_jobs=-1):
    # Fit the classifier to the training data
    mnb_grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print('Best parameters: ', mnb_grid_search.best_params_)

# Use the best parameters to train the model on the full train set
best_mnb = MultinomialNB(**mnb_grid_search.best_params_)
best_mnb.fit(X_train, y_train)

# Predict on the validation and test sets
mnb_y_test_pred = best_mnb.predict(X_test)
mnb_y_val_pred = best_mnb.predict(X_val)

# Evaluate the classifier on the test and validation set
mnb_test_accuracy = best_mnb.score(X_test, y_test)
mnb_validation_accuracy = best_mnb.score(X_val, y_val)

print('Test Accuracy: ', round(mnb_test_accuracy, 2))
print('Validation Accuracy: ', round(mnb_validation_accuracy, 2))

Best parameters:  {'alpha': 1, 'fit_prior': True}
Test Accuracy:  0.68
Validation Accuracy:  0.69


In [5]:
print(classification_report(y_test, mnb_y_test_pred, zero_division=0))
print("Confusion Matrix:", confusion_matrix(y_test, mnb_y_test_pred))

              precision    recall  f1-score   support

     neutral       0.67      0.95      0.79       575
    positive       0.71      0.20      0.31       332

    accuracy                           0.68       907
   macro avg       0.69      0.58      0.55       907
weighted avg       0.69      0.68      0.62       907

Confusion Matrix: [[548  27]
 [265  67]]


### SVM

In [6]:
# Define the parameter grid for hyperparameter tuning
svm_param_grid = {
    'C': [0.1, 1, 10], 
    'kernel': ['linear','rbf']
}

# Create a repeated 5-fold cross-validation object
kf = KFold(n_splits=5)

# Create an SVM classifier
svm = SVC()

# Use GridSearchCV for repeated 5-fold cross validation and hyperparameter tuning
svm_grid_search = GridSearchCV(svm, svm_param_grid, cv=kf, n_jobs=-1)

with parallel_backend('multiprocessing', n_jobs=-1):
    # Fit the classifier to the training data
    svm_grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print('Best parameters: ', svm_grid_search.best_params_)

# Use the best parameters to train the model on the full train set
best_svm = SVC(**svm_grid_search.best_params_)
best_svm.fit(X_train, y_train)

# Predict on the validation and test sets
svm_y_test_pred = best_svm.predict(X_test)
svm_y_val_pred = best_svm.predict(X_val)

# Evaluate the classifier on the test and validation set
svm_test_accuracy = best_svm.score(X_test, y_test)
svm_validation_accuracy = best_svm.score(X_val, y_val)

print('Test Accuracy: ', round(svm_test_accuracy, 2))
print('Validation Accuracy: ', round(svm_validation_accuracy, 2))

Best parameters:  {'C': 1, 'kernel': 'linear'}
Test Accuracy:  0.71
Validation Accuracy:  0.71


In [7]:
print(classification_report(y_test, svm_y_test_pred, zero_division=0))
print("Confusion Matrix:", confusion_matrix(y_test, svm_y_test_pred))

              precision    recall  f1-score   support

     neutral       0.72      0.90      0.80       575
    positive       0.69      0.40      0.51       332

    accuracy                           0.71       907
   macro avg       0.71      0.65      0.65       907
weighted avg       0.71      0.71      0.69       907

Confusion Matrix: [[515  60]
 [199 133]]


### RF

In [8]:
# Define the parameter grid for hyperparameter tuning
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a repeated 5-fold cross-validation object
kf = KFold(n_splits=5)

# Initialize the Random Forest classifier
rf = RandomForestClassifier()

# Use GridSearchCV for repeated 5-fold cross validation and hyperparameter tuning
rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=kf, n_jobs=-1)

with parallel_backend('multiprocessing', n_jobs=-1):
    # Fit the classifier to the training data
    rf_grid_search.fit(X_train, y_train)

# Print the best parameters and score
print('Best parameters: ', rf_grid_search.best_params_)

# Use the best parameters to train the model on the full train set
best_rf = RandomForestClassifier(**rf_grid_search.best_params_)
best_rf.fit(X_train, y_train)

# Predict on the validation and test sets
rf_y_test_pred = best_rf.predict(X_test)
rf_y_val_pred = best_rf.predict(X_val)

# Evaluate the classifier on the test and validation set
rf_test_accuracy = best_rf.score(X_test, y_test)
rf_validation_accuracy = best_rf.score(X_val, y_val)

print('Test Accuracy: ', round(rf_test_accuracy, 2))
print('Validation Accuracy: ', round(rf_validation_accuracy, 2))

Best parameters:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Test Accuracy:  0.73
Validation Accuracy:  0.74


In [9]:
# Print the accuracy and confusion matrix for the validation dataset
print(classification_report(y_test, rf_y_test_pred, zero_division=0))
print("Confusion Matrix:", confusion_matrix(y_test, rf_y_test_pred))

              precision    recall  f1-score   support

     neutral       0.74      0.90      0.81       575
    positive       0.72      0.45      0.55       332

    accuracy                           0.73       907
   macro avg       0.73      0.67      0.68       907
weighted avg       0.73      0.73      0.72       907

Confusion Matrix: [[517  58]
 [183 149]]


### Model Accuracies with Best Hyperparameters

In [10]:
mnb_results = {
    'Model': 'NB',
    'Best parameters': [mnb_grid_search.best_params_],
    'Test Accuracy': [round(mnb_test_accuracy, 2)],
}

svm_results = {
    'Model': 'SVM',
    'Best parameters': [svm_grid_search.best_params_],
    'Test Accuracy': [round(svm_test_accuracy, 2)],
}

rf_results = {
    'Model': 'RF',
    'Best parameters': [rf_grid_search.best_params_],
    'Test Accuracy': [round(rf_test_accuracy, 2)],
}


mnb_report = pd.DataFrame(mnb_results)
svm_report = pd.DataFrame(svm_results)
rf_report = pd.DataFrame(rf_results)

classification_results = pd.concat([mnb_report, svm_report, rf_report], ignore_index=True)
classification_results.to_csv('../4_Results/bert_results.csv', index=False)
classification_results.head()

Model                                    Best parameters  Test Accuracy
0    NB                    {'alpha': 1, 'fit_prior': True}           0.68
1   SVM                       {'C': 1, 'kernel': 'linear'}           0.71
2    RF  {'max_depth': None, 'min_samples_leaf': 1, 'mi...           0.73

### Confusion Matrix Heatmaps

In [12]:
mnb_cm = confusion_matrix(y_test, mnb_y_test_pred)
svm_cm = confusion_matrix(y_test, svm_y_test_pred)
rf_cm = confusion_matrix(y_test, rf_y_test_pred)

labels = ['neutr', 'pos']

fig, axes = plt.subplots(nrows=3, figsize=(8, 12))

sns.heatmap(mnb_cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 15}, xticklabels=labels, yticklabels=labels, cbar=False, ax=axes[0])
axes[0].set_title("Naive Bayes")
axes[0].set_xlabel("Predicted Labels")
axes[0].set_ylabel("True Labels")
axes[0].xaxis.tick_top()
axes[0].xaxis.set_label_position('top')

sns.heatmap(svm_cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 15}, xticklabels=labels, yticklabels=labels, cbar=False, ax=axes[1])
axes[1].set_title("Support Vector Machine")
axes[1].set_xlabel("Predicted Labels")
axes[1].set_ylabel("True Labels")
axes[1].xaxis.tick_top()
axes[1].xaxis.set_label_position('top')

sns.heatmap(rf_cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 15}, xticklabels=labels, yticklabels=labels, cbar=False, ax=axes[2])
axes[2].set_title("Random Forest")
axes[2].set_xlabel("Predicted Labels")
axes[2].set_ylabel("True Labels")
axes[2].xaxis.tick_top()
axes[2].xaxis.set_label_position('top')

plt.tight_layout()
plt.show()
